In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime, date

#### Feed the downloaded file here

In [73]:
df = pd.read_csv(r'C:\Users\user\Downloads\transactions2.csv')

In [74]:
df.columns = df.columns.str.strip()
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2023-12-31,499219379894,Nippon India Liquid Growth Direct Plan,sell,1.724,5799.9769,5799.9769,10000.00
1,2023-12-26,91018778532,Axis ELSS Tax Saver Growth Direct Plan,sell,24.109,87.1531,88.6224,2101.15
2,2023-12-26,2104123675,Invesco India ELSS Tax Saver Growth Direct Plan,sell,12.746,117.6900,118.8800,1500.00
3,2023-12-18,2104123675,Invesco India ELSS Tax Saver Growth Direct Plan,sell,117.000,117.0400,118.8800,13693.54
4,2023-12-18,91018778532,Axis ELSS Tax Saver Growth Direct Plan,sell,32.200,87.7642,88.6224,2825.98


In [75]:
df["Date"] = pd.to_datetime(df["Date"])

In [76]:
df2 = df

#### Prevent all the liquid fund from showing

In [77]:
df = df[~df['Name of the Fund'].str.contains("Kuvera|Liquid|Money|Gilt|Cash|Bond|Debt|Short|Gold|Securities|Duration|Income")]
##remove the upper line in case performance of all the mutual fund is required
df = df.loc[~df['Order'].str.contains("sell")]

In [78]:
today = pd.to_datetime(date.today())

#### Control the number of days to look behind / calculate the Xirr for. for three years we take 365*3 = 1095 days

In [79]:
df = df.loc[df['Date'] > (pd.to_datetime(date.today() - timedelta(days = 1095)))]

In [80]:
mf_name = df['Name of the Fund'].unique()

#### See the list of all the mutual funds

In [81]:
mf_name

array(['Quant ELSS Tax Saver Growth Direct Plan',
       'Mirae Asset ELSS Tax Saver Growth Direct Plan',
       'Bank Of India ELSS Tax Saver Growth Direct Plan',
       'Invesco India ELSS Tax Saver Growth Direct Plan',
       'Axis ELSS Tax Saver Growth Direct Plan',
       'Motilal Oswal ELSS Tax Saver Growth Direct Plan'], dtype=object)

In [82]:
df['Amount (INR)'] = df['Amount (INR)']*-1

In [83]:
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
9,2023-12-01,5102600564,Quant ELSS Tax Saver Growth Direct Plan,buy,3.077,324.9875,355.6267,-999.95
13,2023-11-01,5102600564,Quant ELSS Tax Saver Growth Direct Plan,buy,3.395,294.5402,355.6267,-999.95
17,2023-10-04,5102600564,Quant ELSS Tax Saver Growth Direct Plan,buy,3.351,298.4250,355.6267,-999.95
29,2023-09-01,5102600564,Quant ELSS Tax Saver Growth Direct Plan,buy,3.353,298.2122,355.6267,-999.95
44,2023-07-20,5102600564,Quant ELSS Tax Saver Growth Direct Plan,buy,17.389,287.5225,355.6267,-4999.75


#### Create the  function that calculates Xirr ( funciton copied from https://github.com/SunilVeeravalli/XIRR_in_Python/blob/main/xirr.py )

In [84]:
min_rate = 0
max_rate = 0
def npv(seq_of_rates: np.array, data: pd.DataFrame) -> tuple:
    global min_rate
    global max_rate
    
    for a_rate in seq_of_rates:
        max_date = data['Date'].max()
        data['npv'] = data['Amount'] * ((1 + (a_rate / 100)) ** ((max_date - data['Date']).dt.days / 365))
        
        if data['Amount'].sum() > 0:
            if data['npv'].sum() > 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
        else:
            if data['npv'].sum() < 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
    
    return min_rate, max_rate


def xirr(data: pd.DataFrame) -> float:
    # Finding out which column contains Date and Amount
    try:
        data.iloc[:, 0].astype(float)
        col_names = ['Amount', 'Date']
    except Exception as _:
        col_names = ['Date', 'Amount']
    
    # Renaming the columns accordingly and converting to correct data types
    data.columns = col_names
    data = data.assign(Date = pd.to_datetime(arg = data['Date'], infer_datetime_format = True, dayfirst = True).dt.date,
                       Amount = data['Amount'].astype(float))
    
    if data['Amount'].sum() > 0:
        step_values = [100, 10, 1, 0.01, 0.001, 0.0001]
        stop = 10000
    else:
        step_values = [-100, -10, -1, -0.01, -0.001, -0.0001]
        stop = -10000
    
    start = 0
    for i in range(len(step_values)):
        seq_of_rates = np.arange(start = start, stop = stop, step = step_values[i])
        start, stop = npv(seq_of_rates, data)
    
    return (start + stop) / 2


In [85]:
solution = []
mf= []
for i in range(0,len(mf_name)):
    mf = df[df['Name of the Fund'] == mf_name[i]]
    mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }), 
                   ignore_index = True, sort = True)
    ready_df = mf[['Date','Amount (INR)']]
    solution.append(xirr(ready_df))

C:\Users\user\AppData\Local\Temp\ipykernel_3104\4125528385.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }),
C:\Users\user\AppData\Local\Temp\ipykernel_3104\4125528385.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }),
C:\Users\user\AppData\Local\Temp\ipykernel_3104\4125528385.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }),
C:\Users\user\AppData\Local\Temp\ipykernel_3104\4125528385.py:5: FutureWarning:

In [86]:
final_df = pd.DataFrame(list(zip(mf_name.tolist(),solution)))

In [87]:
final_df.columns = ['Name of the Fund', 'Xirr']

### See the final performance of the mutual fund sorted based on the overall Xirr

In [88]:
final_df.sort_values('Xirr', ascending = False)

,Name of the Fund,Xirr
0,Quant ELSS Tax Saver Growth Direct Plan,25.59505
5,Motilal Oswal ELSS Tax Saver Growth Direct Plan,22.05185
1,Mirae Asset ELSS Tax Saver Growth Direct Plan,18.66605
2,Bank Of India ELSS Tax Saver Growth Direct Plan,17.85885
3,Invesco India ELSS Tax Saver Growth Direct Plan,17.35915
4,Axis ELSS Tax Saver Growth Direct Plan,10.71265


In [89]:
final_df.sort_values('Name of the Fund', ascending = True)

,Name of the Fund,Xirr
4,Axis ELSS Tax Saver Growth Direct Plan,10.71265
2,Bank Of India ELSS Tax Saver Growth Direct Plan,17.85885
3,Invesco India ELSS Tax Saver Growth Direct Plan,17.35915
1,Mirae Asset ELSS Tax Saver Growth Direct Plan,18.66605
5,Motilal Oswal ELSS Tax Saver Growth Direct Plan,22.05185
0,Quant ELSS Tax Saver Growth Direct Plan,25.59505


In [90]:
final_df

,Name of the Fund,Xirr
0,Quant ELSS Tax Saver Growth Direct Plan,25.59505
1,Mirae Asset ELSS Tax Saver Growth Direct Plan,18.66605
2,Bank Of India ELSS Tax Saver Growth Direct Plan,17.85885
3,Invesco India ELSS Tax Saver Growth Direct Plan,17.35915
4,Axis ELSS Tax Saver Growth Direct Plan,10.71265
5,Motilal Oswal ELSS Tax Saver Growth Direct Plan,22.05185


In [91]:
df_temp = df.loc[df['Name of the Fund']=='Quant Large & Mid Cap Growth Direct Plan']

In [92]:
df_temp.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)


In [93]:
df_temp.sort_values(by=['Date'],axis=0, ascending=True, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_3104\3455194109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.sort_values(by=['Date'],axis=0, ascending=True, inplace=True)


In [94]:
df_temp

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)


In [95]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name of the Fund  6 non-null      object 
 1   Xirr              6 non-null      float64
dtypes: float64(1), object(1)
memory usage: 228.0+ bytes
